<a href="https://colab.research.google.com/github/shaking54/compression/blob/master/Shannon_Fano_Coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from collections import Counter,defaultdict
import math
import pandas as pd
import copy
import os

In [10]:
class Shannon_FanoTree():
    def __init__(self):
        #assert data is None
        self.data = []
        self.frequency = []
        self.dict = defaultdict(list)
        self.B1 = 0
        self.B0 = 0
        self.ideal_entropy = 0
        self.encode = ''
    
    # add data to dict
    def add_node(self,name,code):
        f = self.dict[name][-1]
        pi = f/len(self.data)
        lencode = len(code)
        entropy = math.log(1/pi,2)
        self.dict[name].append(entropy)
        self.dict[name].append(code)
        self.dict[name].append(lencode*f)
        self.B1 += f*lencode
        self.ideal_entropy += pi*entropy
    
    """
        recursive [L,R] and code
        go to left => code + '0'
        go to right => code + '1'
    """ 
    def recur(self,Range,code):
        l, r = Range
        if l == r: return 
        if l + 1 == r:
            self.add_node(self.frequency[l][0],code)
            return
        
        idx = self.break_point(Range)
        self.recur((l,idx),code + '0')
        self.recur((idx,r),code + '1')
        
    # function compress data
    def compress(self,data):
        #assign data
        self.data = list(data)
        self.B0 = len(self.data)*8
        
        # get frequency of each elements in data
        frequency = Counter(self.data)
        self.frequency = frequency.most_common()
        
        # insert  
        for key, val in self.frequency:
            self.dict[key].append(val)
            
        self.recur((0,len(self.frequency)),'')
        self.encode = ' '.join([self.dict[x][2] for x in self.data])    
        
    # find idx such that diff sum([L,idx)) and sum([idx,R)) is minimize 
    def break_point(self,Range):
        L, R = Range
        Sum = 0
        # Calc Sum in [L,R]
        for i in range(L,R):
            _,f = self.frequency[i]
            Sum += f
            
        mindiff = 9999999999999
        part1 = 0
        breakp = L + 1
        for i in range(L,R):
            _,f = self.frequency[i]
            part1 += f
            diff = abs(Sum - 2*part1)
            if diff < mindiff:
                mindiff = diff
                breakp = i + 1
        return breakp
    
    # sent encode to decoder 
    def share_encode(self):
        return (self.encode,self.dict)
    
    def get_DataFrame(self):
        return pd.DataFrame.from_dict(self.dict, orient='index',
                                columns=['Freq', '-log2(pi)', 'Code', 'Bits used'])
    def get_information(self):
        print(self.get_DataFrame())
        print('\nB0                  : {}'.format(self.B0))
        print('B1                  : {}'.format(self.B1))
        print('Compression Ratio   : {}'.format(self.B0/self.B1))
        print('Ideal Entropy       : {}'.format(self.ideal_entropy))
        print('Compression Entropy : {}'.format((8*self.B1)/self.B0))
        
    @staticmethod
    def decoder(encode):
        code,encode_dict = encode
        decode_dict = dict([(val[2],key) for key,val in encode_dict.items()])
        return ''.join(list(map(lambda x: decode_dict[x],code.split())))

In [2]:
import zipfile

zipfile.ZipFile('/content/drive/My Drive/data.zip', 'r').extractall('/content/')

In [35]:
f = open('/content/data/text/1.txt', 'r')
data = f.read()
test = Shannon_FanoTree()
test.compress(data)
test.get_information()

    Freq  -log2(pi)        Code  Bits used
      97   2.766409         000        291
e     51   3.693897         001        153
s     35   4.237039        0100        140
a     34   4.278859        0101        136
i     33   4.321928        0110        132
r     33   4.321928       01110        165
o     33   4.321928       01111        165
t     30   4.459432       10000        150
l     25   4.722466       10001        125
n     23   4.842760       10010        115
c     18   5.196397       10011         90
u     16   5.366322       10100         80
p     16   5.366322      101010         96
h     13   5.665882      101011         78
T     13   5.665882       10110         65
m     12   5.781360      101110         72
y     10   6.044394      101111         60
d     10   6.044394      110000         60
S      9   6.196397      110001         54
\n     9   6.196397      110010         54
R      9   6.196397      110011         54
P      8   6.366322      110100         48
'      8   

In [32]:
array = []

In [33]:
for filename in os.listdir('/content/data/text'):
    if filename.endswith('.txt'):
        data = open('/content/data/text/' + filename).read()
        encoder = Shannon_FanoTree()
        encoder.compress(data)
        array.append([encoder.B0, encoder.B1, round(encoder.B0/encoder.B1,5), round(encoder.ideal_entropy,5), round((8*encoder.B1)/encoder.B0,5)])

In [34]:
from prettytable import PrettyTable
t = PrettyTable(['Bits_Before', 'Bit_After', 'Compression Ratio', 'Ideal_Entropy', 'Compression Entropy'])
for i in array:
    t.add_row(i)
print(t)

+-------------+-----------+-------------------+---------------+---------------------+
| Bits_Before | Bit_After | Compression Ratio | Ideal_Entropy | Compression Entropy |
+-------------+-----------+-------------------+---------------+---------------------+
|     6024    |    3702   |      1.62723      |    4.86545    |       4.91633       |
|     4848    |    3035   |      1.59736      |    4.95382    |       5.00825       |
|     4824    |    3012   |      1.60159      |    4.94418    |       4.99502       |
|     3600    |    2200   |      1.63636      |    4.86094    |       4.88889       |
|     3752    |    2314   |      1.62143      |    4.88342    |        4.9339       |
|     5592    |    3387   |      1.65102      |    4.81698    |       4.84549       |
|     6896    |    4278   |      1.61197      |    4.90341    |       4.96288       |
|     7144    |    4446   |      1.60684      |    4.92814    |       4.97872       |
|     4848    |    2969   |      1.63287      |    4.8